# TELL Runs for IM3's Experiment Group B 

This notebook executes the initial set of runs of the TELL model for IM3's Experiment Group B.

In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell


## Set the Directory Structure

In [5]:
# Identify the top-level directory and the subdirectory where the data will be stored:
current_dir =  '/Users/burl878/OneDrive - PNNL/Documents/IMMM/Data/TELL/Production_Runs'
tell_data_dir = os.path.join(current_dir, r'tell_data')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)


## Run the MLP Models

In [ ]:
# Generate a list of BA abbreviations to process:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()

scenario_to_process = 'rcp85hotter_ssp5'

# Run the MLP prediction step for the list of BAs using parallel processing streams:
for year_to_process in range(2020,2030,1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                             year = year_to_process,
                             data_dir = os.path.join(tell_data_dir, r'wrf_to_tell', r'wrf_tell_bas_output', scenario_to_process),
                             datetime_field_name = 'Time_UTC',
                             save_prediction = True,
                             prediction_output_directory = os.path.join(tell_data_dir, r'outputs', r'mlp_output', scenario_to_process),
                             n_jobs = -1)


## Run the Forward Execution

In [11]:
# Run the TELL model forward in time for a given year:
summary_df, ba_time_series_df, state_time_series_df = tell.execute_forward(year_to_process = '2025',
                                                                           gcam_target_year = '2025', 
                                                                           scenario_to_process = 'rcp85hotter_ssp5',
                                                                           data_input_dir = tell_data_dir,
                                                                           save_county_data = False)


Start time =  2022-10-12 15:37:42.103631


FileNotFoundError: [Errno 2] No such file or directory: '/Users/burl878/OneDrive - PNNL/Documents/IMMM/Data/TELL/Production_Runs/tell_data/sample_forcing_data/sample_gcam_usa_data/gcamDataTable_aggParam.csv'

## 5. Model visualization

The final section of this quickstarter notebook plots some of the output of `tell` to give the user a flavor of what the model is doing. Note that the sample output data in the `tell` quickstarter covers the years 2039, 2059, 2079, and 2099 so those are the only valid values for the `year_to_plot` variable in each function call.

### 5.1. Plot the state annual total loads from GCAM-USA and `tell`

The first visualization plots the annual total loads from both GCAM-USA and `tell`. The data plotted here are in units of TWh and the `tell` values are the unscaled projections. The scaled projections `tell` are by definition equal to those from GCAM-USA.

In [ ]:
# Plot the annual total loads from both GCAM-USA and TELL:
tell.plot_state_annual_total_loads(year_to_plot = '2039',
                                   gcam_target_year = '2039',
                                   scenario_to_plot = 'rcp85hotter_ssp5',  
                                   data_input_dir = tell_data_dir,
                                   image_output_dir = tell_image_dir,
                                   image_resolution = 150,
                                   save_images = True)


### 5.2. Plot the time-series of total hourly loads for a given state

Here we plot time-series of the raw (unscaled) and scaled total loads from `tell` at the state level. The user specifies which state they want to plot using the `state_to_plot" variable in the function call.

In [ ]:
# Plot the time-series of raw and scaled loads from TELL at the state level for a user-specified state:
tell.plot_state_load_time_series(state_to_plot = 'Connecticut', 
                                 year_to_plot = '2039',
                                 gcam_target_year = '2039',
                                 scenario_to_plot = 'rcp85hotter_ssp5', 
                                 data_input_dir = tell_data_dir,
                                 image_output_dir = tell_image_dir,
                                 image_resolution = 150,
                                 save_images = True)


### 5.3. Plot the load duration curve for a given state

Our last plot at the state level is the load duration curve which shows the frequency at which a given load occurs in a state. The user specifies which state they want to plot using the "state_to_plot" variable in the function call.

In [ ]:
# Plot the load duration curve at the state level for a user-specified state:
tell.plot_state_load_duration_curve(state_to_plot = 'North Carolina', 
                                    year_to_plot = '2039',
                                    gcam_target_year = '2039',
                                    scenario_to_plot = 'rcp85hotter_ssp5', 
                                    data_input_dir = tell_data_dir,
                                    image_output_dir = tell_image_dir,
                                    image_resolution = 150,
                                    save_images = True)


### 5.4. Plot the time-series of total hourly loads for a given BA

Our final visualization plots the time-series of the raw (unscaled) and scaled total loads from `tell` at the BA level. The user specifies which BA they want to plot using the "ba_to_plot" variable in the function call.

In [ ]:
# Plot the time-series of raw and scaled loads from TELL at the BA level for a user-specified BA (e.g., PJM, CISO, ERCO, etc.):
tell.plot_ba_load_time_series(ba_to_plot = 'NYIS', 
                              year_to_plot = '2039',
                              gcam_target_year = '2039',
                              scenario_to_plot = 'rcp85hotter_ssp5', 
                              data_input_dir = tell_data_dir,
                              image_output_dir = tell_image_dir,
                              image_resolution = 150,
                              save_images = True)
